In [ ]:

from keras.layers import Input
from keras.optimizers import Adam
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import json
import tensorflow as tf
import os


from model import UNet
from DataGen import DataGen
from utils import *

%load_ext autoreload
%autoreload 2

#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
## drive/Mydrive should be added to beggining of all paths if you want to execute from the drive path
train_path = ".\\uavid_train\\*\\Images\\"
train_label_path= ".\\label_train\\"
val_path = ".\\uavid_val\\*\\Images\\"
val_label_path= ".\\label_val\\"
IMAGE_SIZE=128
BATCH_SIZE=16
EPOCHS=70

loaded_model_path=f"./model-{IMAGE_SIZE}-{BATCH_SIZE}-{EPOCHS}.h5"

In [ ]:
model = UNet(IMAGE_SIZE)
train_gen = DataGen(train_path, train_label_path, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
valid_gen = DataGen(val_path, val_label_path, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
callback=tf.keras.callbacks.ModelCheckpoint(
    "./model-128.h5",
    monitor= 'val_loss',
    verbose= 1,
    save_freq='epoch')
model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["acc"])
model.summary()

In [ ]:
history=model.fit(train_gen, validation_data=valid_gen, 
                steps_per_epoch=train_gen.steps, validation_steps=valid_gen.steps, 
                callbacks=[callback],epochs=EPOCHS)

with open('file.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:
model.save("model.h5")

#plot loss
history_dict = json.load(open("file.json", 'r'))
plt.plot(history_dict['val_loss'], label = "val_loss")
plt.plot(history_dict['loss'], label = "loss")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
#plot accuracy
plt.plot(history_dict['val_acc'], label = "val_acc")
plt.plot(history_dict['acc'], label = "acc")
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
#if you want to load already trained and saved model.
loaded_model=load_model(loaded_model_path)

In [ ]:
#predict validation datas and plot confusion matrix and show photos
predict(train_gen,loaded_model,image_size=IMAGE_SIZE)

In [ ]:

plot_all_conf(train_gen,loaded_model,image_size=IMAGE_SIZE)